In [2]:
from parser import extract_json, extract_contents
import time

### Soybean oil

ID No. 946

### Vitamin K1 epoxide-1,4-diol

ID No. 23496

Each food has:
- all existing attributes from original data
- list of Compounds.

Each compound has: (possibly several contents per food-compound relation)
- all existing attributes from original data
- original content min
- original content max
- original content avg
- original unit
- citation (reference)

In [3]:
time_start = time.time()
content = extract_contents()
time_end = time.time()
print("Runtime of extracting contents (seconds):", time_end - time_start)

Runtime of extracting contents (seconds): 43.5323212146759


In [4]:
content[(946, 23496)]

{'orig_contents': {'min': 82.35,
  'max': 90.95,
  'avg': 86.65,
  'unit': 'mg/100 g'},
 'reference': 'USDA'}

In [5]:
compound = extract_json('Compound.json')

In [6]:
vitaminK = compound[23496].copy()
vitaminK['orig_contents'] = content[(946, 23496)]['orig_contents']
vitaminK['reference'] = content[(946, 23496)]['reference']
vitaminK

{'name': 'Vitamin K1 epoxide-1,4-diol',
 'state': 'Liquid',
 'annotation_quality': 'low',
 'description': 'Vitamin K is a family of fat-soluble compounds with a common chemical structure, 2, methyl-1,4-napthoquinone. Phylloquinone is present in food of plant origin, such as green, leafy vegetables and certain plant oils, and is the predominant form in the diet. Bacterial and other forms of vitamin K, referred to as the menaquinones, differ in structure from phylloquinone in their 3-substituted lipophilic side chain. Menaquinone-4 (MK-4), which is alkylated from menadione, is present in animal feeds or is the product of tissue-specific conversion directly from dietary phylloquinone. Vitamin K is a cofactor specific to the formation of gamma-carboxyglutamyl (Gla) residues in certain proteins, including prothrombin necessary for normal hemostatic function. The naturally occurring forms of vitamin K are quinones (i.e. phylloquinone and menaquinones) so vitamin K is reduced to the vitamin K

In [7]:
food = extract_json('Food.json')

In [8]:
soybean_oil = food[946]
soybean_oil.setdefault('compounds', []).append(vitaminK)
soybean_oil

{'name': 'Soybean oil',
 'name_scientific': None,
 'description': 'Soybean oil is a vegetable oil extracted from the seeds of the soybean (Glycine max). It is one of the most widely consumed cooking oils. As a drying oil, processed soybean oil is also used as a base for printing inks (soy ink) and oil paints.',
 'itis_id': None,
 'wikipedia_id': None,
 'picture_file_name': 'Soybean_oil.jpg',
 'picture_content_type': 'image/jpeg',
 'picture_file_size': 78352,
 'picture_updated_at': '2020-04-17T16:22:50.000Z',
 'legacy_id': None,
 'food_group': 'Herbs and Spices',
 'food_subgroup': 'Oilseed crops',
 'food_type': 'Type 2',
 'created_at': '2020-01-24T17:05:36.000Z',
 'updated_at': '2020-04-17T16:22:51.000Z',
 'creator_id': None,
 'updater_id': None,
 'export_to_afcdb': False,
 'category': None,
 'ncbi_taxonomy_id': None,
 'export_to_foodb': True,
 '_id': 'FOOD00914',
 'compounds': [{'name': 'Vitamin K1 epoxide-1,4-diol',
   'state': 'Liquid',
   'annotation_quality': 'low',
   'description

In [9]:
start_time = time.time()
for food_id, compound_id in content.keys():
    if not content[(food_id, compound_id)]: continue
    try:
        compound_item = compound[compound_id].copy()
        compound_item['orig_contents'] = content[(food_id, compound_id)]['orig_contents']
        compound_item['reference'] = content[(food_id, compound_id)]['reference']
        food_item = food[food_id]
        food_item.setdefault('compounds', []).append(compound_item)
    except KeyError:
        continue
end_time = time.time()
print("Runtime of creating compound lists for each food item (seconds):", end_time - start_time)

Runtime of creating compound lists for each food item (seconds): 0.23506522178649902


In [10]:
food[946]['compounds'].pop(0)
food[946]

{'name': 'Soybean oil',
 'name_scientific': None,
 'description': 'Soybean oil is a vegetable oil extracted from the seeds of the soybean (Glycine max). It is one of the most widely consumed cooking oils. As a drying oil, processed soybean oil is also used as a base for printing inks (soy ink) and oil paints.',
 'itis_id': None,
 'wikipedia_id': None,
 'picture_file_name': 'Soybean_oil.jpg',
 'picture_content_type': 'image/jpeg',
 'picture_file_size': 78352,
 'picture_updated_at': '2020-04-17T16:22:50.000Z',
 'legacy_id': None,
 'food_group': 'Herbs and Spices',
 'food_subgroup': 'Oilseed crops',
 'food_type': 'Type 2',
 'created_at': '2020-01-24T17:05:36.000Z',
 'updated_at': '2020-04-17T16:22:51.000Z',
 'creator_id': None,
 'updater_id': None,
 'export_to_afcdb': False,
 'category': None,
 'ncbi_taxonomy_id': None,
 'export_to_foodb': True,
 '_id': 'FOOD00914',
 'compounds': [{'name': 'Zinc',
   'state': 'Solid',
   'annotation_quality': 'low',
   'description': 'Although zinc is an 